In [1]:
import warnings
warnings.filterwarnings("ignore")

### 怎样让我们自己创建的对象可以使用for in进行遍历

#### getitem方法，每次for in 获取数据时，都会调用这个方法，优先级低

一个对象是可迭代对象，肯定可以通过 for in 遍历

能通过for in 进行遍历的，不一定是可迭代对象，比如有 getitem:

In [2]:
class Person:
    def __init__(self):
        self.result = 1

    def __getitem__(self, item):   # 需要有终止条件
        self.result += 1
        if self.result >= 6:
            raise StopIteration("停止遍历") 

        return self.result

p = Person()

for i in p:
    print(i)

2
3
4
5


#### 优先级: iter > getitem

In [3]:
class Person:
    def __init__(self):
        self.result = 1

    def __getitem__(self, item):
        print("getitem")
        if self.result >= 6:
            raise StopIteration("停止遍历")
        return self.result

    def __iter__(self):
        print("iter")

p = Person()

for i in p:
    print(i)

iter


TypeError: iter() returned non-iterator of type 'NoneType'

In [4]:
l = [1, 2, 3, 4]
iter(l)

In [5]:
class Person:
    def __init__(self):
        self.result = 1

    def __iter__(self):
        print("iter")
        return iter([1, 2, 3, 4, 5])  # 这里是列表，可迭代

p = Person()

for i in p:
    print(i)

iter
1
2
3
4
5


In [6]:
class Person:
    def __init__(self):
        self.result = 1

    def __iter__(self):
        print("iter")
        return self    # 不可迭代，迭代器需要有next方法

p = Person()

for i in p:
    print(i)

iter


TypeError: iter() returned non-iterator of type 'Person'

#### 一个“迭代器”，需实现 iter 和 next 2个方法

#### 会调用iter方法返回的迭代器对象的next方法

In [10]:
class Person:
    def __init__(self):
        self.age = 1

    def __iter__(self):    # step1 iter 
        print("iter")
        return self
    
    def __next__(self):    # step2 next
        self.age += 1
        if self.age >= 6:
            raise StopIteration("停止遍历")
        return self.age

p = Person()

for i in p:
    print(i)
    
import collections

# Iterator需要 iter 和 next 两个
print(isinstance(p, collections.Iterator))

iter
2
3
4
5
True


一个迭代器肯定是一个可迭代对象，反之不一定

一个对象是可迭代对象，肯定可以通过 for in / next 遍历，反之不一定

In [11]:
class Person:
    def __init__(self):
        self.result = 1

    def __iter__(self):    # step1 iter 
        print("iter")
        return self

p = Person()

# Iterable 只需要 iter 一个
print(isinstance(p, collections.Iterable))

True


#### 怎样使我们自己创建的对象可以使用next函数进行访问

#### 使用next方法

In [8]:
class Person:
    def __init__(self):
        self.result = 1
    
    def __next__(self):    # next，注意设定终止条件
        self.result += 1
        if self.result >= 6:
            raise StopIteration("停止遍历")
        return self.result

p = Person()
print(next(p))

2


综上:一个迭代器肯定可以用next方法访问，但必须同时具备next和iter才是迭代器，  
即能用next方法访问的不一定是迭代器，只具备iter方法的是可迭代对象。

#### 注意： iter方法可以恢复迭代器的初始化值，复用迭代器

In [18]:
class Person:
    def __init__(self):
        self.age = 1

    def __iter__(self):
        self.age = 1  # 初始化值，复用迭代器
        return self

    def __next__(self):
        self.age += 1
        if self.age >= 6:
            raise StopIteration("停止遍历")
        return self.age


p = Person()

for i in p:
    print(i)

for i in p:
    print(i)

2
3
4
5
2
3
4
5


#### 使用iter()

In [15]:
pm = iter(p)
print(pm is p)  # iter() 就是
for i in pm:
    print(i)

True
2
3
4
5


In [12]:
class Person:
    def __init__(self):
        self.age = 1
    def __getitem__(self, item):
        self.age += 1
        if self.age >= 6:
            raise StopIteration("stop")
        return self.age

p = Person()
pt = iter(p)
print(pt)
# 没有 getitem就无法生成

In [13]:
for i in pt:
    print(i)

2
3
4
5


In [21]:
class Person:
    def __init__(self):
        self.age = 1

    def __iter__(self):
        self.age = 1  
        return self

    def __next__(self):
        self.age += 1
        if self.age >= 6:
            raise StopIteration("停止遍历")
        return self.age


p = Person()

ps = iter(p, 4)    # 此时不行
for i in ps:
    print(i)

TypeError: iter(v, w): v must be callable

In [20]:
class Person:
    def __init__(self):
        self.age = 1

    def __iter__(self):
        self.age = 1  
        return self

    def __next__(self):
        self.age += 1
        if self.age >= 6:
            raise StopIteration("停止遍历")
        return self.age


p = Person()

ps = iter(p.__next__, 4)    # 注意这里
for i in ps:
    print(i)

2
3


In [22]:
class Person:
    def __init__(self):
        self.age = 1

    def __iter__(self):
        self.age = 1  
        return self

    def __call__(self):  # 加了 call 方法，删了 next
        self.age += 1
        if self.age >= 6:
            raise StopIteration("停止遍历")
        return self.age


p = Person()

ps = iter(p, 4)    # 注意这里
for i in ps:
    print(i)

2
3
